In [1]:
import tdc
import torch
import torch_geometric
import pandas as pd
import rdkit.Chem
import numpy as np
from tdc import single_pred
import tdc

In [2]:
from PyDRP.Data.transfer.drugs import ToxRicPreprocessingPipeline, TransferDrugsDatasetManager, TDCSingleInstanceWrapper, MultiTaskPreprocessingPipeline, MakeDrugwise
from PyDRP.Data.features.targets import MultitargetMinMaxScaling
from PyDRP.Data.features.drugs import GraphCreator
from PyDRP.Data.utils import TorchGraphsTransferDataset
from PyDRP.Data import NI60

In [3]:
ppls = [TDCSingleInstanceWrapper(single_pred.Tox(name="Tox21", label_name = "NR-AR")),
       TDCSingleInstanceWrapper(single_pred.Tox(name="Tox21", label_name = "NR-AhR")),
       MakeDrugwise(NI60()),
       ToxRicPreprocessingPipeline()]

Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!


In [4]:
manager = TransferDrugsDatasetManager(drugs_processing_pipeline = MultiTaskPreprocessingPipeline(ppls),
                           target_processor = MultitargetMinMaxScaling(),
                           drug_featurizer = GraphCreator(),
                           k = 25)

In [ ]:
drug_dict = manager.get_drugs()

In [6]:
train, test, val = manager.get_partition(1)
train_dataset = TorchGraphsTransferDataset(train, drug_dict)
val_dataset = TorchGraphsTransferDataset(val, drug_dict)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 512,
                                               collate_fn = torch_geometric.data.Batch.from_data_list, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = 512,
                                               collate_fn = torch_geometric.data.Batch.from_data_list)